In [ ]:
# install depedencies
%pip install yfinance==0.1.67
%pip install pandas==1.3.3
#%pip install requests==2.26.0
%pip install beautifulsoup4
%pip install plotly==5.3.1
%pip install html5lib
%pip install --upgrade nbformat

In [2]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns

In [3]:
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    stock_data_specific = stock_data[stock_data.Date <= '2021--06-14']
    revenue_data_specific = revenue_data[revenue_data.Date <= '2021-04-30']
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific.Date, infer_datetime_format=True), y=stock_data_specific.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data_specific.Date, infer_datetime_format=True), y=revenue_data_specific.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=1200,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()

In [4]:
tsla = yf.Ticker("TSLA")
tsla.info

{'zip': '78725',
 'sector': 'Consumer Cyclical',
 'fullTimeEmployees': 99290,
 'longBusinessSummary': 'Tesla, Inc. designs, develops, manufactures, leases, and sells electric vehicles, and energy generation and storage systems in the United States, China, and internationally. The company operates in two segments, Automotive, and Energy Generation and Storage. The Automotive segment offers electric vehicles, as well as sells automotive regulatory credits. It provides sedans and sport utility vehicles through direct and used vehicle sales, a network of Tesla Superchargers, and in-app upgrades; and purchase financing and leasing services. This segment is also involved in the provision of non-warranty after-sales vehicle services, sale of used vehicles, retail merchandise, and vehicle insurance, as well as sale of products to third party customers; services for electric vehicles through its company-owned service locations, and Tesla mobile service technicians; and vehicle limited warrantie

In [25]:
urlTsla = 'https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue'
resTsla = requests.get(urlTsla).text
soupTsla = BeautifulSoup(resTsla,'html.parser')
soupTsla.find_all('title')
tsla_revenue_data = pd.DataFrame(columns=["Date", "Revenue"])
for row in soupTsla.find_all("tbody")[1].find_all('tr'):
    td = row.find_all("td")
    new_row = pd.DataFrame({"Date": [td[0].text],"Revenue": [td[1].text]})
    tsla_revenue_data = pd.concat((tsla_revenue_data, new_row))
tsla_revenue_data["Revenue"] = tsla_revenue_data["Revenue"].str.replace(',|\$',"")
tsla_revenue_data = tsla_revenue_data[tsla_revenue_data['Revenue'] != ""]
tsla_revenue_data.tail()

C:\Users\u071574\AppData\Local\Temp\ipykernel_22432\2168280569.py:10: FutureWarning:

The default value of regex will change from True to False in a future version.



,Date,Revenue
0,2010-09-30,31
0,2010-06-30,28
0,2010-03-31,21
0,2009-09-30,46
0,2009-06-30,27


In [6]:
gme = yf.Ticker("GME")
gme.info

{'zip': '76051',
 'sector': 'Consumer Cyclical',
 'fullTimeEmployees': 12000,
 'longBusinessSummary': 'GameStop Corp., a specialty retailer, provides games and entertainment products through its e-commerce properties and various stores in the United States, Canada, Australia, and Europe. The company sells new and pre-owned gaming platforms; accessories, such as controllers, gaming headsets, virtual reality products, and memory cards; new and pre-owned gaming software; and in-game digital currency, digital downloadable content, and full-game downloads. It also sells collectibles comprising licensed merchandise primarily related to the gaming, television, and movie industries, as well as pop culture themes. As of January 29, 2022, the company operated 4,573 stores and ecommerce sites under the GameStop, EB Games, and Micromania brands; and 50 pop culture themed stores that sell collectibles, apparel, gadgets, electronics, toys, and other retail products under the Zing Pop Culture brand, 

In [24]:
urlGme = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html'
resGme = requests.get(urlGme)
soupGme = BeautifulSoup(resGme.content,'html5lib')
gme_revenue_data = pd.DataFrame(columns=["Date", "Revenue"])
for row in soupGme.find_all("tbody")[1].find_all('tr'):
    td = row.find_all("td")
    new_row = pd.DataFrame({"Date": [td[0].text],"Revenue": [td[1].text]})
    gme_revenue_data = pd.concat((gme_revenue_data, new_row))
gme_revenue_data["Revenue"] = gme_revenue_data["Revenue"].str.replace(',|\$',"")
gme_revenue_data = gme_revenue_data[gme_revenue_data['Revenue'] != ""]
gme_revenue_data.tail()

C:\Users\u071574\AppData\Local\Temp\ipykernel_22432\1110963369.py:9: FutureWarning:

The default value of regex will change from True to False in a future version.



,Date,Revenue
0,2006-01-31,1667
0,2005-10-31,534
0,2005-07-31,416
0,2005-04-30,475
0,2005-01-31,709


In [27]:
tsla_share_price_data = tsla.history(period="max")
tsla_share_price_data.reset_index(inplace=True)
make_graph(tsla_share_price_data, tsla_revenue_data,'Tesla')

In [28]:
gme_share_price_data = gme.history(period="max")
gme_share_price_data.reset_index(inplace=True)
make_graph(gme_share_price_data, gme_revenue_data,'Gamestop')

# end